In [1]:
from constants import *

paper = True # 

In [72]:
# install alpaca-py
! python3 -m pip install alpaca-py

Defaulting to user installation because normal site-packages is not writeable


In [73]:
! python3 -m pip install polygon

Defaulting to user installation because normal site-packages is not writeable


In [45]:
import polygon


# test polygon
stocks_client = polygon.StocksClient(polygon_api_key)
previous_close = stocks_client.get_previous_close('AMD')
print(previous_close)

{'ticker': 'AMD', 'queryCount': 1, 'resultsCount': 1, 'adjusted': True, 'results': [{'T': 'AMD', 'v': 44482655.0, 'vw': 155.0104, 'o': 153.1, 'c': 155.78, 'h': 156.65, 'l': 151.26, 't': 1715025600000, 'n': 397279}], 'status': 'OK', 'request_id': 'babd0de64a1c163bf736b885d251772f', 'count': 1}


In [46]:
import alpaca
from alpaca.data.live.stock import *
from alpaca.data.historical.stock import *
from alpaca.data.requests import *
from alpaca.data.timeframe import *
from alpaca.trading.client import *
from alpaca.trading.stream import *
from alpaca.trading.requests import *
from alpaca.trading.enums import *
from alpaca.common.exceptions import APIError

# alpaca_url = "https://paper-api.alpaca.markets/v2/assets?attributes="
# headers = {
#     "accept": "application/json",
#     "APCA-API-KEY-ID": "PKK2TFCB5V938IEVR6MS",
#     "APCA-API-SECRET-KEY": "7bgRpzUXYJKL5pGKAvDsMG726fd4OC4gPfHbFddW"
# }

# response = requests.get(alpaca_url, headers=headers)

# print(response.text)

In [47]:
# to run async code in jupyter notebook
import nest_asyncio
nest_asyncio.apply()

In [48]:
# check version of alpaca-py
alpaca.__version__

'0.21.0'

In [78]:
! python3 -m pip install ta

Defaulting to user installation because normal site-packages is not writeable


In [49]:
import requests
import time
from ta import trend
import numpy as np
from datetime import datetime, timedelta
from pytz import timezone
from zoneinfo import ZoneInfo
import json

In [50]:
# Below are the variables for development this documents
# Please do not change these variables
trade_api_url = None
trade_api_wss = None
data_api_url = None
stream_data_wss = None

alpaca_trade_client = TradingClient(api_key=alpaca_api_key, secret_key=alpaca_secret_key, paper=paper, url_override=trade_api_url)

acct = alpaca_trade_client.get_account()
acct

# setup clients

# not used
# from alpaca.data import StockHistoricalDataClient
# trade_client = StockHistoricalDataClient(api_key=alpaca_api_key, secret_key=alpaca_secret_key)

{   'account_blocked': False,
    'account_number': 'PA3S7VFUNZTF',
    'accrued_fees': '0',
    'buying_power': '199999.1',
    'cash': '99999.55',
    'created_at': datetime.datetime(2024, 5, 1, 10, 55, 41, 614181, tzinfo=TzInfo(UTC)),
    'crypto_status': <AccountStatus.ACTIVE: 'ACTIVE'>,
    'currency': 'USD',
    'daytrade_count': 0,
    'daytrading_buying_power': '0',
    'equity': '99999.55',
    'id': UUID('6df96e60-3440-4b3a-98e8-c2d1192e44dd'),
    'initial_margin': '0',
    'last_equity': '99999.55',
    'last_maintenance_margin': '0',
    'long_market_value': '0',
    'maintenance_margin': '0',
    'multiplier': '2',
    'non_marginable_buying_power': '99999.55',
    'options_approved_level': 2,
    'options_buying_power': '99999.55',
    'options_trading_level': 2,
    'pattern_day_trader': False,
    'pending_transfer_in': '0',
    'pending_transfer_out': None,
    'portfolio_value': '99999.55',
    'regt_buying_power': '199999.1',
    'short_market_value': '0',
    'shor

In [55]:
session = requests.session()

# We only consider stocks with per-share prices inside this range
min_share_price = 2.0
max_share_price = 13.0
# Minimum previous-day dollar volume for a stock we might consider
min_last_dv = 1000000
# Stop limit to default to
default_stop = .97
# How much of our portfolio to allocate to any one position
risk = 0.001
momentum_percentage_needed = 5

In [56]:
# def get_1000m_history_data(symbols):
#     print('Getting historical data...')
#     minute_history = {}
#     c = 0
#     for symbol in symbols:
#         minute_history[symbol] = api.polygon.historic_agg(
#             size="minute", symbol=symbol, limit=1000
#         ).df
#         c += 1
#         print('{}/{}'.format(c, len(symbols)))
#     print('Success.')
#     return minute_history

# stocks_client.get_aggregate_bars(timespan="minute", symbol="AMD", limit=1000, from_date="2024-01-01", to_date="2024-05-03")

symbolz = ['AMD', 'NVIDIA']

def get_1000m_history_data(symbols):
    print('Getting historical data...')
    minute_history = {}
    c = 0
    for symbol in symbols:
        minute_history[symbol] = stocks_client.get_aggregate_bars(
            timespan="minute", symbol=symbol, limit=1000, from_date="2024-01-01", to_date="2024-05-03"
        )
        c += 1
        print('{}/{}'.format(c, len(symbols)))
    print('Success.')
    return minute_history

get_1000m_history_data(symbolz)

Getting historical data...
1/2
2/2
Success.


{'AMD': {'ticker': 'AMD',
  'queryCount': 1000,
  'resultsCount': 1000,
  'adjusted': True,
  'results': [{'v': 2978,
    'vw': 147.8374,
    'o': 147.78,
    'c': 147.63,
    'h': 148.29,
    'l': 147.62,
    't': 1704186000000,
    'n': 90},
   {'v': 997,
    'vw': 147.5108,
    'o': 147.45,
    'c': 147.45,
    'h': 147.45,
    'l': 147.45,
    't': 1704186060000,
    'n': 33},
   {'v': 2211,
    'vw': 147.7134,
    'o': 147.7,
    'c': 147.72,
    'h': 147.72,
    'l': 147.7,
    't': 1704186180000,
    'n': 37},
   {'v': 953,
    'vw': 147.5489,
    'o': 147.59,
    'c': 147.51,
    'h': 147.59,
    'l': 147.51,
    't': 1704186240000,
    'n': 19},
   {'v': 264,
    'vw': 147.5748,
    'o': 147.56,
    'c': 147.56,
    'h': 147.56,
    'l': 147.56,
    't': 1704186360000,
    'n': 17},
   {'v': 140,
    'vw': 147.6,
    'o': 147.6,
    'c': 147.6,
    'h': 147.6,
    'l': 147.6,
    't': 1704186480000,
    'n': 3},
   {'v': 214,
    'vw': 147.7865,
    'o': 147.8,
    'c': 147.8,

In [57]:
# def get_tickers():
#     print('Getting current ticker data...')
#     tickers = api.polygon.all_tickers()
#     print('Success.')
#     assets = api.list_assets()
#     symbols = [asset.symbol for asset in assets if asset.tradable]
#     return [ticker for ticker in tickers if (
#         ticker.ticker in symbols and
#         ticker.lastTrade['p'] >= min_share_price and
#         ticker.lastTrade['p'] <= max_share_price and
#         ticker.prevDay['v'] * ticker.lastTrade['p'] > min_last_dv and
#         ticker.todaysChangePerc >= 3.5
#     )]

def get_tickers():
    print('Getting current ticker data...')
    tickers = stocks_client.get_snapshot_all()
    tickers = tickers['tickers'] # returns a dict with key tickers
    print('tickers length', len(tickers), tickers[0])
    assets = alpaca_trade_client.get_all_assets()
    symbols = [asset.symbol for asset in assets if asset.tradable]
    print ('symbols', symbols)
    tradeable_symbols = []

    for single_ticker_data in tickers:
        if (single_ticker_data['ticker'] in symbols and 
        single_ticker_data['prevDay']['o'] >= min_share_price and
        single_ticker_data['prevDay']['o'] <= max_share_price and
        single_ticker_data['prevDay']['v'] * single_ticker_data['prevDay']['c'] > min_last_dv and
        single_ticker_data['todaysChangePerc'] >= momentum_percentage_needed): 
            tradeable_symbols.append(single_ticker_data['ticker'])
    
    print('tradeable', len(tradeable_symbols))

get_tickers()

Getting current ticker data...
tickers length 11024 {'ticker': 'PSC', 'todaysChangePerc': 0, 'todaysChange': 0, 'updated': 0, 'day': {'o': 0, 'h': 0, 'l': 0, 'c': 0, 'v': 0, 'vw': 0}, 'min': {'av': 0, 't': 0, 'n': 0, 'o': 0, 'h': 0, 'l': 0, 'c': 0, 'v': 0, 'vw': 0}, 'prevDay': {'o': 48.22, 'h': 48.47, 'l': 48.22, 'c': 48.41, 'v': 8164, 'vw': 48.4078}}
symbols ['SPXC', 'SPXT', 'ETB', 'SPXU', 'FLQS', 'SPXV', 'FLRG', 'SPY', 'FLRT', 'SPYV', 'FLSA', 'FLSP', 'FLSW', 'FLTB', 'FLTR', 'FLTW', 'FLUD', 'SPYX', 'FLV', 'SQ', 'FLXS', 'FLYD', 'FLYU', 'FLYX.WS', 'SPWO', 'FMAG', 'SPXX', 'FMAR', 'SPYT', 'FMAY', 'FMB', 'AES', 'ET.PRI', 'FMCX', 'FMDE', 'FMED', 'ET.PRE', 'FMF', 'FMHI', 'FMN', 'AER', 'EMR', 'CHN', 'FMNY', 'FMQQ', 'FRPT', 'FMST', 'FMSTW', 'SPXE', 'FMY', 'SPXL', 'SPXN', 'SRDX', 'SRE', 'SRET', 'SRG', 'SERV', 'SREA', 'ESUS', 'TRIB', 'FNGD', 'FNGO', 'FROG', 'CNXT', 'FNGU', 'SRHBY', 'AYI', 'FNVT', 'FNVTU', 'FNVTW', 'FNWB', 'FNWD', 'ESQ', 'APCA', 'FOA.WS', 'FOCT', 'FOF', 'ESP', 'FRO', 'ESOA', 'SRI

In [2]:
def find_stop(current_value, minute_history, now):
    series = minute_history['low'][-100:] \
                .dropna().resample('5min').min()
    series = series[now.floor('1D'):]
    diff = np.diff(series.values)
    low_index = np.where((diff[:-1] <= 0) & (diff[1:] > 0))[0] + 1
    if len(low_index) > 0:
        return series[low_index[-1]] - 0.01
    return current_value * default_stop

In [8]:
! python3 -m pip install pydantic
! python3 -m pip install asyncio  
! python3 -m pip install websockets

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.8/101.8 kB 4.1 MB/s eta 0:00:00
Defaulting to user installation because normal site-packages is not writeable


In [39]:
import json
import queue
from typing import Optional, Dict, Callable, Union
import asyncio
import websockets
import logging

from pydantic import BaseModel

from alpaca.common import RawData
from alpaca.common.enums import BaseURL
from alpaca.trading import TradeUpdate

log = logging.getLogger(__name__)


class TradingStream:
    """
    This is a WebSocket client which allows you to streaming data from your trading account.

    Learn more here: https://alpaca.markets/docs/api-references/trading-api/streaming/

    If using paper keys, make sure to set ``paper`` to True when instantiating the client.
    """

    def __init__(
        self,
        alpaca_api_key: str,
        alpaca_secret_key: str,
        paper: bool = True,
        raw_data: bool = False,
        url_override: str = None,
        websocket_params: Optional[Dict] = None,
    ):
        self._api_key = alpaca_api_key
        self._secret_key = alpaca_secret_key
        self._trade_updates_handler = None
        self._endpoint = (
            url_override
            if url_override
            else BaseURL.TRADING_STREAM_PAPER if paper else BaseURL.TRADING_STREAM_LIVE
        )
        self._ws = None
        self._running = False
        self._loop = None
        self._raw_data = raw_data
        self._stop_stream_queue = queue.Queue()
        self._should_run = True
        self._websocket_params = {
            "ping_interval": 10,
            "ping_timeout": 180,
            "max_queue": 1024,
        }

        if websocket_params:
            self._websocket_params = websocket_params

    async def _connect(self):
        self._ws = await websockets.connect(self._endpoint, **self._websocket_params)

    async def _auth(self):
        await self._ws.send(
            json.dumps(
                {
                    "action": "authenticate",
                    "data": {
                        "key_id": self._api_key,
                        "secret_key": self._secret_key,
                    },
                }
            )
        )
        r = await self._ws.recv()
        msg = json.loads(r)
        if msg.get("data").get("status") != "authorized":
            raise ValueError("failed to authenticate")

    async def _dispatch(self, msg: Dict) -> None:
        """Distributes message from websocket connection to appropriate handler

        Args:
            msg (Dict): The message from the websocket connection
        """
        stream = msg.get("stream")
        if stream == "trade_updates":
            if self._trade_updates_handler:
                await self._trade_updates_handler(self._cast(msg))

    def _cast(self, msg: Dict) -> Union[BaseModel, RawData]:
        """Parses data from websocket message if raw_data is False, otherwise
        returns raw websocket message

        Args:
            msg (Dict): The message containing market data

        Returns:
            Union[BaseModel, RawData]: The raw or parsed live data
        """
        result = msg
        if not self._raw_data:
            result = TradeUpdate(**msg.get("data"))
        return result

    async def _subscribe_trade_updates(self) -> None:
        if self._trade_updates_handler:
            await self._ws.send(
                json.dumps({"action": "listen", "data": {"streams": ["trade_updates"]}})
            )

    def subscribe_trade_updates(self, handler: Callable):
        """
        Subscribes to trade updates for your trading account.

        Args:
            handler (Callable): The async handler that will receive trade update data.

        Returns:
            None
        """
        self._ensure_coroutine(handler)
        self._trade_updates_handler = handler
        if self._running:
            asyncio.run_coroutine_threadsafe(
                self._subscribe_trade_updates(), self._loop
            ).result()

    async def _start_ws(self):
        await self._connect()
        await self._auth()
        log.info(f"connected to: {self._endpoint}")
        await self._subscribe_trade_updates()

    async def _consume(self):
        while True:
            if not self._stop_stream_queue.empty():
                self._stop_stream_queue.get(timeout=1)
                await self.close()
                break
            else:
                try:
                    r = await asyncio.wait_for(self._ws.recv(), 5)
                    msg = json.loads(r)
                    await self._dispatch(msg)
                except asyncio.TimeoutError:
                    # ws.recv is hanging when no data is received. by using
                    # wait_for we break when no data is received, allowing us
                    # to break the loop when needed
                    pass

    async def _run_forever(self):
        self._loop = asyncio.get_running_loop()
        # do not start the websocket connection until we subscribe to something
        while not self._trade_updates_handler:
            if not self._stop_stream_queue.empty():
                self._stop_stream_queue.get(timeout=1)
                return
            await asyncio.sleep(0.1)
        log.info("started trading stream")
        self._should_run = True
        self._running = False
        while True:
            try:
                if not self._should_run:
                    log.info("Trading stream stopped")
                    return
                if not self._running:
                    log.info("starting trading websocket connection")
                    await self._start_ws()
                    self._running = True
                    await self._consume()
            except websockets.WebSocketException as wse:
                await self.close()
                self._running = False
                log.warning(
                    "trading stream websocket error, restarting "
                    + " connection: "
                    + str(wse)
                )
            except Exception as e:
                log.exception(
                    "error during websocket " "communication: {}".format(str(e))
                )
            finally:
                await asyncio.sleep(0.01)

    async def close(self) -> None:
        """Closes the websocket connection."""
        if self._ws:
            await self._ws.close()
            self._ws = None
            self._running = False

    async def stop_ws(self) -> None:
        """Signals websocket connection should close by adding a closing message to the stop_stream_queue"""
        self._should_run = False
        if self._stop_stream_queue.empty():
            self._stop_stream_queue.put_nowait({"should_stop": True})

    def stop(self) -> None:
        """Stops the websocket connection."""
        if self._loop.is_running():
            asyncio.run_coroutine_threadsafe(self.stop_ws(), self._loop).result()

    async def run(self) -> None:
        """Starts up the websocket connection's event loop"""
        try:
            await self._run_forever() # jupyter
            print('running in run')
            # asyncio.run(self._run_forever()) # Ipython
        except KeyboardInterrupt:
            print("keyboard interrupt, bye")
            pass

    def _ensure_coroutine(self, handler: Callable) -> None:
        """Checks if a method is an asyncio coroutine method

        Args:
            handler (Callable): A method to be checked for coroutineness

        Raises:
            ValueError: Raised if the input method is not a coroutine
        """
        if not asyncio.iscoroutinefunction(handler):
            raise ValueError("handler must be a coroutine function")

In [42]:
tradestream = TradingStream(alpaca_api_key, alpaca_secret_key)



running in run


/var/folders/yz/q7bfk6350tg_mczsbvvk5mbr0000gn/T/ipykernel_51400/2362499486.py:207: RuntimeWarning: coroutine 'TradingStream._run_forever' was never awaited
  self._run_forever() # jupyter


RuntimeError: This event loop is already running

In [ ]:
conn.send()